# Building a Spam Filter with Naive Bayes

#### Darren Ho

## Introduction

In this project, we're going to study the practical side of the multinomial Naive Bayes algorithm by building a spam filter for SMS messages. 

To classify messages as spam or non-spam, we need the computer to: 

- Learn how humans classify messages
- Use the human knowledge to estimate probabilities for new message - proabilities for spam and non-spam
- Classify a new message based on these probability values - if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if values are equal, then we may need a human to classify the message)

For this project, our goal is to create a spam filter that classifies new messages with accuracy greater than 80%. 

## Exploring the Dataset

Our first task is to "teach" the computer how to classify messages. To do so, we will use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans. 

The dataset was put together by Tiago A. Almeida and Jose maria Gomez Hidalgo, and it can be downloaded from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

Let's explore the dataset.

In [1]:
# read in data

import pandas as pd

# data points are tab separated so sep=\t , no header row so header=none, and names parameter to 
# name columns

text_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

print('Number of Rows:',text_spam.shape[0])
print('Number of Columns:',text_spam.shape[1])

Number of Rows: 5572
Number of Columns: 2


In [2]:
text_spam.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
text_spam['Label'].value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

In [4]:
round(text_spam['Label'].value_counts(normalize=True)*100,3)

ham     86.594
spam    13.406
Name: Label, dtype: float64

We see that the `text_spam` dataset has 5,572 rows and only 2 columns: `Label` and `SMS`. Then we see that out of the 5,572 messages, 4,825 of them are classified as `ham` (which means non-spam) while the remaining 747 are classified as `spam`. In percentage form, approximately 87% of the messages are `ham`, while the remaining 13% are `spam`. 

## Training and Test Set

Now that we've explored the dataset a bit, lets split our dataset into two categories:

- A **training set**, which we'll use to "train" the computer how to classify messages
- A **test set**, which we'll use to test how good the spam filter is with classifying new messages

We're going to retain 80% of our dataset for training, and the remaining 20% for testing. With 5,572 messages in the dataset:

- The training set will have 4,458 messages
- The test set will have 1,114 messages

In [5]:
# randomizing entire dataset
# frac to randomize dataset, random_state to make sure results are reproducible

sample_spam = text_spam.sample(frac=1, random_state=1)  

# splitting into training and test set
# reset index labels for both sets - labels remained unordered after randomization

training_spam = sample_spam[:4458].reset_index(drop=True)
test_spam = sample_spam[4458:].reset_index(drop=True)

print('Training Set:', training_spam.shape)
print('Test Set:', test_spam.shape)

Training Set: (4458, 2)
Test Set: (1114, 2)


In [6]:
training_spam['Label'].value_counts(normalize=True)*100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [7]:
test_spam['Label'].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

We started by randomizing the entire dataset to ensure that `spam` and `ham` messages are spread properly throughout the dataset.

For the `training_spam` dataset, we found that approximately 87% of the messages were labeled `ham` and the remaining 13% were labeled as `spam`. For the `test_spam` dataset, we found that approximately 87% of the messages were labeled `ham` and the remaining 13% were labeled as `spam`.

The percentages are similar to what we have in the full dataset. 

## Letter Case and Punctuation

We need to perform a bit of data cleaning to transform the data into a format that will allow us to extract needed information easily, so that we are able to calculate probabilities that help with classifying messages. 

In [8]:
training_spam.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [9]:
# remove punctuation and then transforms every letter in every word to lower case

training_spam['SMS'] = training_spam['SMS'].str.replace('\W', ' ').str.lower()
training_spam.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [10]:
training_spam.tail()

,Label,SMS
4453,ham,sorry i ll call later in meeting any thing re...
4454,ham,babe i fucking love you too you know fuck...
4455,spam,u ve been selected to stay in 1 of 250 top bri...
4456,ham,hello my boytoy geeee i miss you already a...
4457,ham,wherre s my boytoy


## Creating the Vocabulary

We want to transform the `SMS` column into many columns such that every other column represents a unique word in our vocabulary (more specifically, each column shows the frequency of that unique word for any given message).

We'll eventually bring the training set to that format ourselves, by first, let's create a list with all the unique words that occur in the messages of our training set. 

In [11]:
# transform each message into a list by splitting the string at the space character
training_spam['SMS'] = training_spam['SMS'].str.split()

# initiate empty list
vocabulary = []

# appending every word of every message to vocab list
for msg in training_spam['SMS']:
    for word in msg:
        vocabulary.append(word)

# transform vocabulary into a set which removes duplicates
# transform set back into a list 
vocabulary = list(set(vocabulary))

In [12]:
print('Unique Words:', len(vocabulary))
print(vocabulary[:10])

Unique Words: 7783
['arngd', 'steed', 'womdarfull', '09064018838', 'matured', '4041', 'wth', 'qing', 'wined', 'hme']


## The Final Training Set

Now we're going to use the `vocabulary` to make the data transformation we need. We'll first build a dictionary that we'll then convert to the dataframe we need. 

In [13]:
# dictionary where each key is unique word from vocab df, each value is a list of the length of
# training set, where each element in the list is a 0
word_counts_per_sms = {unique_word: [0] * len(training_spam['SMS']) for unique_word in vocabulary}

# we loop to get both index and SMS message
for index, sms in enumerate(training_spam['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [14]:
# transform dictionary to dataframe

word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [15]:
# concatenate the df we just built with training_spam

training_set = pd.concat([training_spam, word_counts], axis=1)
training_set.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating Constants First

We can begin creating the spam filter now that we are done with cleaning the data and have a training set to work with.

Recall that the Naive Bayes algorithm will need to know the probability values of the two equations to be able to classify new messages: 

$$\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}$$

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, recall that we need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Let's calculate:

- P(Spam) & P(Ham)
- N (Spam), N (Ham), N (Vocabulary)

    - Where N (Spam) is equal to the number of words in all the spam messages
    - Where N (Ham) is equal to the number of words in all the ham messages

We'll also use Laplace smoothing and set alpha to equal 1. 

In [16]:
# filtering training set into spam versus ham
spam = training_set[training_set['Label'] == 'spam']
ham = training_set[training_set['Label'] == 'ham']

# P(Spam) versus P(Ham)
p_spam = len(spam) / len(training_set)
p_ham = len(ham) / len(training_set)

print('P(Spam):', p_spam)
print('P(Ham):', p_ham)

P(Spam): 0.13458950201884254
P(Ham): 0.8654104979811574


In [17]:
# N_Spam
n_spam = spam['SMS'].apply(len).sum()

# N_Ham
n_ham = ham['SMS'].apply(len).sum()

# N_Vocabulary
n_vocab = len(vocabulary)

# Laplace smoothing
alpha = 1

print('N(Spam):', n_spam)
print('N(Ham):', n_ham)
print('N(Vocabulary):', n_vocab)
print('Alpha:', alpha)


N(Spam): 15190
N(Ham): 57237
N(Vocabulary): 7783
Alpha: 1


## Calculating Parameters

Let's now calculate all the parameters using the equations below: 

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [18]:
# dictionary to store the parameters for P(w|Spam) and P(w|ham)
spam_parameters = {unique_word:0 for unique_word in vocabulary}
ham_parameters = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    # calculate Nw|spam
    n_word_given_spam = spam[word].sum()    # sums column w/ name matching word
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocab)     # calculate probability word given spam
    spam_parameters[word] = p_word_given_spam           # add to spam parameters 
    
    n_word_given_ham = ham[word].sum()   
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocab)
    ham_parameters[word] = p_word_given_ham


## Classifying A New Message

Now that we have calculated the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in input as a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help

In [21]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # calculate the probability that a message is spam and ham
    p_spam_given_message = p_spam 
    p_ham_given_message = p_ham
    
    for word in message:
        # if word is present in dict containing spam params, update value by multiplying w/
        # param value specific to that word
        if word in spam_parameters:
            p_spam_given_message *= spam_parameters[word]
        
        # for ham params
        if word in ham_parameters:
            p_ham_given_message *= ham_parameters[word]
            
        # we ignore words if they are not in vocabulary

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [22]:
# testing function above

classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [23]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's Accuracy

We'll now try to determine how well the spam filter does on our test set of 1,114 messages that we set aside at the beginning. The algorithm will output a label for every message in our test set, which we will be able to compare with the actual label (given by a human).  

In [24]:
# tweaking function above so that it returns statements instead of printing

def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in spam_parameters:
            p_spam_given_message *= spam_parameters[word]

        if word in ham_parameters:
            p_ham_given_message *= ham_parameters[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set. 

In [25]:
test_spam['predicted'] = test_spam['SMS'].apply(classify_test_set)
test_spam.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we can compare the predicted values with the actual values to measure how good the spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:

\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}

In [26]:
# measuring accuracy of spam filter
correct = 0
total = test_spam.shape[0]

for row in test_spam.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


In [27]:
(test_spam['Label'] == test_spam['predicted']).value_counts(normalize=True)

True     0.987433
False    0.012567
dtype: float64

We see that the spam filter we created has 98.7% accuracy when it comes to classifying messages as `spam` or `ham`. We doubled checked the accuracy by creating a boolean mask where `True` meant that the `Label` was equal to what the spam filter `predicted`, and then calculated the percentage of `True` and `False` in the test set.   

## Conclusion

In this project, we were able to study the practical side of the multinomial Naive Bayes algorithm by creating a spam filter for SMS messages. We were able to classify messages as spam or non-spam by teaching the computer to: 

- Learn how humans classify messages
- Use the human knowledge to estimate probabilities for new message - proabilities for spam and non-spam
- Classify a new message based on these probability values

Our goal was to create a spam filter that classifies new messages with accuracy greater than 80%. We were able to blow that number out the water as the filter we created had an accuracy of 98.74% on the test set. 

## Next Steps

Here's a few next steps we can take to improve upon the project:

- Isolate the 14 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions
- Make the filtering process more complex by making the algorithm sensitive to letter case